# A Smarter Approach: Semi-Automated Feature Engineering

The manual feature engineering approach is extremely inefficient. It often involves writing the same exact code over and over again to do very similar results mostly dealing with grouping, aggregating, and merging. Anytime we start to repeat code, we have to ask is there a more efficient way to do this? The answer of course is yes as we will shortly see. In this notebook, we will take another approach to feature engineering, one halfway between purely manual and automated: we will call this approach __semi-automated modeling__. 

The idea behind semi-automated modeling is to take many of the repeated actions and encode them in functions where they can be used repeatedly with only the arguments (inputs to the function) changing rather than the syntax itself. Although this still does not allow for complete re-usability across problems, it will create a much more efficient workflow. Moreover, we will be able to create thousands of features, which was not a possibility when we were making features one at a time spending 15 minutes on each feature! 

The general idea of semi-automated feature engineering is picture below:

![](../../images/SemiAutoFeatureEngineering.png)

## Approach

We will develop two primary functions for handling data:

1. `agg_numeric`: calculate statistics (`mean`, `max`, `min`, `sum`, `count`) of the numeric variables across all the children of each parent individual in a child table.
2. `agg_categorical`: calculate the `sum`, `mean`, and `count` for each unique category of each categorical variable across all the children of each parent individual in a child table. 

When we want to completely summarize a child table such as `bureau`, we need to run the first function and then the second, merging the results at the end. We can build on these two functions by writing higher level functions that do these actions for us. Likewise, the grandchildren dataframes require multiple applications of both `agg_numeric` and `agg_categorical`, and we will also write a function to perform all of these operations for us. As a reminder, there are four grandchildren dataframes and two child dataframes. The two higher level functions are:

1. `agg_child`: completely aggregate a child table for each observation of a parent
2. `agg_grandchild`: completely aggregate a grandchild table at the grandparent level. 

Once we have aggregated the six total tables besides `app`, we then merge together all of the data onto `app` by the `SK_ID_CURR` to arrive at one dataframe that can be split back into the training and testing sets. 

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Read in the datasets and replace the anomalous values
app = pd.read_csv('../input/features_manual_domain.csv')
bureau = pd.read_csv('../input/bureau.csv').replace({365243: np.nan})
bureau_balance = pd.read_csv('../input/bureau_balance.csv').replace({365243: np.nan})
cash = pd.read_csv('../input/POS_CASH_balance.csv').replace({365243: np.nan})
credit = pd.read_csv('../input/credit_card_balance.csv').replace({365243: np.nan})
previous = pd.read_csv('../input/previous_application.csv').replace({365243: np.nan})
installments = pd.read_csv('../input/installments_payments.csv').replace({365243: np.nan})

In [4]:
print('Shape of App (originally 121 features): ', app.shape)

Shape of App (originally 121 features):  (356255, 273)


Through manual feature engineering, we added 120 feature from one-hot encoding and then another 30 one at a time.

## Numeric Aggregations

While we were building features one at a time because we wanted to use domain knowledge, if we take the position that we no longer care about the phyiscal representation of the feature, we can easily make hundreds of features that are not limited by our imagination and supposed knowledge. We will call this process __semi-automated feature engineering__ because while it is not automated all the way, we use functions that can repeatedly carry out multiple operations for us without requiring us to manually type them in every time. This will allow for much more efficient workflows and we can still encode domain knowledge by adding features one at a time in addition to the numerous feature from the semi-automated method. 

For example, we limited ourselves to doing aggregations one at a time and using only counts, sums, and means. However, there is nothing stopping us from using other aggregations and performing them all at once. As an implementation of this idea, the code below finds the `count`, `mean`, `max`, `min`, and `sum` of all the numeric columns in the `bureau` dataframe.

In [5]:
bureau_agg = bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR').agg(['count', 'mean', 'max', 'min', 'sum'])
bureau_agg.head()

DAYS_CREDIT                                     CREDIT_DAY_OVERDUE  \
                 count         mean    max     min     sum              count   
SK_ID_CURR                                                                      
100001.0             7  -735.000000  -49.0 -1572.0 -5145.0                  7   
100002.0             8  -874.000000 -103.0 -1437.0 -6992.0                  8   
100003.0             4 -1400.750000 -606.0 -2586.0 -5603.0                  4   
100004.0             2  -867.000000 -408.0 -1326.0 -1734.0                  2   
100005.0             3  -190.666667  -62.0  -373.0  -572.0                  3   

                                 ...    DAYS_CREDIT_UPDATE                     \
           mean  max  min  sum   ...                 count        mean    max   
SK_ID_CURR                       ...                                            
100001.0    0.0  0.0  0.0  0.0   ...                     7  -93.142857   -6.0   
100002.0    0.0  0.0  0.0  0.0   ...                     8 -499.875000   -7.0   
100003.0    0.0  0.0  0.0  0.0   ...                     4 -816.000000  -43.0   
100004.0    0.0  0.0  0.0  0.0   ...                     2 -532.000000 -382.0   
100005.0    0.0  0.0  0.0  0.0   ...                     3  -54.333333  -11.0   

                           AMT_ANNUITY                                      
               min     sum       count         mean      max  min      sum  
SK_ID_CURR                                                                  
100001.0    -155.0  -652.0           7  3545.357143  10822.5  0.0  24817.5  
100002.0   -1185.0 -3999.0           7     0.000000      0.0  0.0      0.0  
100003.0   -2131.0 -3264.0           0          NaN      NaN  NaN      0.0  
100004.0    -682.0 -1064.0           0          NaN      NaN  NaN      0.0  
100005.0    -121.0  -163.0           3  1420.500000   4261.5  0.0   4261.5  

[5 rows x 60 columns]

We can rename the columns to keep track of what they represent. We also add in two lines to remove the columns with exactly the same values. This can occur when using the `count` function.

In [6]:
# New column names
columns = []
# Iterate through the variables names
for var in bureau_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in bureau_agg.columns.levels[1]:
            # Make a new column name for the variable and stat
            columns.append('BUREAU_%s_%s' % (var, stat))
            
bureau_agg.columns = columns

# Remove the columns with all redundant values
_, idx = np.unique(bureau_agg, axis = 1, return_index=True)
bureau_agg = bureau_agg.iloc[:, idx]

bureau_agg.head()

,BUREAU_DAYS_CREDIT_count,BUREAU_DAYS_CREDIT_mean,BUREAU_DAYS_CREDIT_max,BUREAU_DAYS_CREDIT_min,BUREAU_DAYS_CREDIT_sum,BUREAU_CREDIT_DAY_OVERDUE_count,BUREAU_CREDIT_DAY_OVERDUE_mean,BUREAU_CREDIT_DAY_OVERDUE_max,BUREAU_CREDIT_DAY_OVERDUE_min,BUREAU_CREDIT_DAY_OVERDUE_sum,...,BUREAU_DAYS_CREDIT_UPDATE_count,BUREAU_DAYS_CREDIT_UPDATE_mean,BUREAU_DAYS_CREDIT_UPDATE_max,BUREAU_DAYS_CREDIT_UPDATE_min,BUREAU_DAYS_CREDIT_UPDATE_sum,BUREAU_AMT_ANNUITY_count,BUREAU_AMT_ANNUITY_mean,BUREAU_AMT_ANNUITY_max,BUREAU_AMT_ANNUITY_min,BUREAU_AMT_ANNUITY_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,7,-735.000000,-49.0,-1572.0,-5145.0,7,0.0,0.0,0.0,0.0,...,7,-93.142857,-6.0,-155.0,-652.0,7,3545.357143,10822.5,0.0,24817.5
100002.0,8,-874.000000,-103.0,-1437.0,-6992.0,8,0.0,0.0,0.0,0.0,...,8,-499.875000,-7.0,-1185.0,-3999.0,7,0.000000,0.0,0.0,0.0
100003.0,4,-1400.750000,-606.0,-2586.0,-5603.0,4,0.0,0.0,0.0,0.0,...,4,-816.000000,-43.0,-2131.0,-3264.0,0,NaN,NaN,NaN,0.0
100004.0,2,-867.000000,-408.0,-1326.0,-1734.0,2,0.0,0.0,0.0,0.0,...,2,-532.000000,-382.0,-682.0,-1064.0,0,NaN,NaN,NaN,0.0
100005.0,3,-190.666667,-62.0,-373.0,-572.0,3,0.0,0.0,0.0,0.0,...,3,-54.333333,-11.0,-121.0,-163.0,3,1420.500000,4261.5,0.0,4261.5


It's hard to say if any of these features are useful but there is no harm (besides the time investment) in at least making the features. Once we have these, we can merge them with the application data and train and cross-validate a new model. 

Once we no longer care about whether these make sense from a domain knowledge perspective, we open up the possibilities for new features. While this may seem like a naive approach, I would say it is more naive to think that the only good features are those that we come up with on our own based on our knowledge. 

## Numeric Aggregation Function

Refactoring code into functions is a great way to increase efficiency and decrease the chance of making an error (once the function is written correctly). We can take this process and put it in a function that can then be applied to any dataset. The following function aggregates all the numeric variables in a child dataframe at the parent level. That is, for each parent, gather together (group) all of their children, and calculate the aggregations statistics across the children. The function also removes any columns that share the exact same values (which might happen using `count`). 

In [7]:
def agg_numeric(df, parent_var, df_name):
    """
    Groups and aggregates the numeric values in a child dataframe
    by the parent variable.
    
    Parameters
    --------
        df (dataframe): 
            the child dataframe to calculate the statistics on
        parent_var (string): 
            the parent variable used for grouping and aggregating
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated by the `parent_var` for 
            all numeric columns. Each observation of the parent variable will have 
            one row in the dataframe with the parent variable as the index. 
            The columns are also renamed using the `df_name`. Columns with all duplicate
            values are removed. 
    
    """
    
    # Remove id variables other than grouping variable
    for col in df:
        if col != parent_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    # Only want the numeric variables
    parent_ids = df[parent_var].copy()
    numeric_df = df.select_dtypes('number').copy()
    numeric_df[parent_var] = parent_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(parent_var).agg(['count', 'mean', 'max', 'min', 'sum'])

    # Need to create new column names
    columns = []

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        if var != parent_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))
    
    agg.columns = columns
    
    # Remove the columns with all redundant values
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    
    return agg

In [8]:
bureau_agg = agg_numeric(bureau, 'SK_ID_CURR', 'BUREAU')
bureau_agg.head()

,BUREAU_DAYS_CREDIT_sum,BUREAU_DAYS_ENDDATE_FACT_sum,BUREAU_DAYS_CREDIT_min,BUREAU_DAYS_CREDIT_ENDDATE_min,BUREAU_DAYS_ENDDATE_FACT_min,BUREAU_DAYS_ENDDATE_FACT_mean,BUREAU_DAYS_CREDIT_mean,BUREAU_DAYS_CREDIT_UPDATE_sum,BUREAU_DAYS_ENDDATE_FACT_max,BUREAU_DAYS_CREDIT_UPDATE_min,...,BUREAU_AMT_CREDIT_SUM_DEBT_mean,BUREAU_AMT_CREDIT_SUM_min,BUREAU_AMT_CREDIT_SUM_mean,BUREAU_AMT_CREDIT_SUM_DEBT_max,BUREAU_AMT_CREDIT_SUM_max,BUREAU_AMT_CREDIT_SUM_DEBT_sum,BUREAU_AMT_CREDIT_SUM_sum,BUREAU_AMT_CREDIT_MAX_OVERDUE_min,BUREAU_AMT_CREDIT_MAX_OVERDUE_mean,BUREAU_AMT_CREDIT_MAX_OVERDUE_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,-5145.0,-3302.0,-1572.0,-1329.0,-1328.0,-825.500000,-735.000000,-652.0,-544.0,-155.0,...,85240.928571,85500.0,207623.571429,373239.0,378000.0,596686.5,1453365.000,NaN,NaN,NaN
100002.0,-6992.0,-4185.0,-1437.0,-1072.0,-1185.0,-697.500000,-874.000000,-3999.0,-36.0,-1185.0,...,49156.200000,0.0,108131.945625,245781.0,450000.0,245781.0,865055.565,0.0,1681.029,5043.645
100003.0,-5603.0,-3292.0,-2586.0,-2434.0,-2131.0,-1097.333333,-1400.750000,-3264.0,-540.0,-2131.0,...,0.000000,22248.0,254350.125000,0.0,810000.0,0.0,1017400.500,0.0,0.000,0.000
100004.0,-1734.0,-1065.0,-1326.0,-595.0,-683.0,-532.500000,-867.000000,-1064.0,-382.0,-682.0,...,0.000000,94500.0,94518.900000,0.0,94537.8,0.0,189037.800,0.0,0.000,0.000
100005.0,-572.0,-123.0,-373.0,-128.0,-123.0,-123.000000,-190.666667,-163.0,-123.0,-121.0,...,189469.500000,29826.0,219042.000000,543087.0,568800.0,568408.5,657126.000,0.0,0.000,0.000


We can then apply this function to the other child of `application`, the `previous` data which has information on each client's previous loans at Home Credit (as opposed to credits at other financial institutions found in `bureau`).
The resulting dataframe can then be joined to the `app` dataframe using the parent variable, `SK_ID_CURR`.

In [9]:
previous_agg = agg_numeric(previous, 'SK_ID_CURR', 'PREVIOUS')
previous_agg.head()

,PREVIOUS_DAYS_DECISION_sum,PREVIOUS_DAYS_DECISION_min,PREVIOUS_DAYS_DECISION_mean,PREVIOUS_DAYS_DECISION_max,PREVIOUS_DAYS_FIRST_DUE_sum,PREVIOUS_DAYS_FIRST_DUE_min,PREVIOUS_DAYS_FIRST_DUE_mean,PREVIOUS_DAYS_FIRST_DUE_max,PREVIOUS_DAYS_LAST_DUE_sum,PREVIOUS_DAYS_LAST_DUE_min,...,PREVIOUS_AMT_APPLICATION_sum,PREVIOUS_DAYS_FIRST_DRAWING_min,PREVIOUS_DAYS_FIRST_DRAWING_mean,PREVIOUS_DAYS_FIRST_DRAWING_max,PREVIOUS_RATE_INTEREST_PRIMARY_min,PREVIOUS_RATE_INTEREST_PRIMARY_mean,PREVIOUS_RATE_INTEREST_PRIMARY_max,PREVIOUS_RATE_INTEREST_PRIVILEGED_min,PREVIOUS_RATE_INTEREST_PRIVILEGED_mean,PREVIOUS_RATE_INTEREST_PRIVILEGED_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,-1740.0,-1740.0,-1740.0,-1740.0,-1709.0,-1709.0,-1709.000000,-1709.0,-1619.0,-1619.0,...,24835.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100002.0,-606.0,-606.0,-606.0,-606.0,-565.0,-565.0,-565.000000,-565.0,-25.0,-25.0,...,179055.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003.0,-3915.0,-2341.0,-1305.0,-746.0,-3823.0,-2310.0,-1274.333333,-716.0,-3163.0,-1980.0,...,1306309.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004.0,-815.0,-815.0,-815.0,-815.0,-784.0,-784.0,-784.000000,-784.0,-724.0,-724.0,...,24282.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005.0,-1072.0,-757.0,-536.0,-315.0,-706.0,-706.0,-706.000000,-706.0,-466.0,-466.0,...,44617.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Aggregating Categorical Variables

The categorical variables from the children tables also need to be incorporated into the main data. With the application data, we one-hot encoded the variables and then used these directly as features. For the children data tables, we can take a similar approach but with an added aggregation step after one-hot encoding to create one row for each parent. In the traditional feature engineering this was done using sums, but again we do not have to limit ourselves to one aggregation (we will use `sum`,`mean`, and `count`). We also can use all of the categorical features at once instead of picking and choosing single columns or values.

As an example of how this works, consider the `bureau` dataframe.

In [12]:
bureau_ids = bureau['SK_ID_CURR'].copy()

# Select only the categorical columns
bureau_categorical = bureau.select_dtypes('object').copy()
bureau_categorical = pd.get_dummies(bureau_categorical)
bureau_categorical.head()

,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,...,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The one-hot-encoded data is now all numeric which means we can use the grouping and aggregating functions. However, here it does not make sense to find the `min` and `max` since these will just be 0 and 1 for binary variables.

In [13]:
bureau_categorical['SK_ID_CURR'] = bureau_ids

# Grouping and aggregating the one hot encoded data
bureau_categorical = bureau_categorical.groupby('SK_ID_CURR').agg(['sum', 'count', 'mean'])

column_names = []
    
# Iterate through the columns in level 
for var in bureau_categorical.columns.levels[0]:
    # Iterate through the stats in level 1
    for stat in ['sum', 'count', 'mean']:
        # Make a new column name
        column_names.append('%s_%s_%s' % ('BUREAU', var, stat))
            
bureau_categorical.columns = column_names

_, idx = np.unique(bureau_categorical, axis = 1, return_index=True)
bureau_categorical = bureau_categorical.iloc[:, idx]

bureau_categorical.head()

,BUREAU_CREDIT_TYPE_Mobile operator loan_mean,BUREAU_CREDIT_TYPE_Mobile operator loan_sum,BUREAU_CREDIT_TYPE_Loan for purchase of shares (margin lending)_mean,BUREAU_CREDIT_TYPE_Loan for purchase of shares (margin lending)_sum,BUREAU_CREDIT_ACTIVE_Bad debt_mean,BUREAU_CREDIT_ACTIVE_Bad debt_sum,BUREAU_CREDIT_TYPE_Interbank credit_mean,BUREAU_CREDIT_TYPE_Interbank credit_sum,BUREAU_CREDIT_TYPE_Real estate loan_mean,BUREAU_CREDIT_TYPE_Real estate loan_sum,...,BUREAU_CREDIT_TYPE_Credit card_sum,BUREAU_CREDIT_ACTIVE_Active_mean,BUREAU_CREDIT_ACTIVE_Closed_mean,BUREAU_CREDIT_TYPE_Consumer credit_mean,BUREAU_CREDIT_CURRENCY_currency 1_mean,BUREAU_CREDIT_ACTIVE_Active_sum,BUREAU_CREDIT_ACTIVE_Closed_sum,BUREAU_CREDIT_TYPE_Consumer credit_sum,BUREAU_CREDIT_CURRENCY_currency 1_sum,BUREAU_CREDIT_ACTIVE_Active_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.428571,0.571429,1.000000,1.0,3,4,7,7,7
100002.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,4,0.250000,0.750000,0.500000,1.0,2,6,4,8,8
100003.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,2,0.250000,0.750000,0.500000,1.0,1,3,2,4,4
100004.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.000000,1.000000,1.000000,1.0,0,2,2,2,2
100005.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1,0.666667,0.333333,0.666667,1.0,2,1,2,3,3


The process above will also be incorporated into a function that can be used with any child dataframe. It has the same inputs as `agg_numeric` but will deal with only the categorical variables. 

## Categorical Aggregation Function

Much like the numerical aggregation function, the `agg_categorical` function works on a child dataframe to aggregate statistics at the parent level. This can work with any child of `app` and might even be extensible to other problems with only minor changes in syntax.

In [14]:
def agg_categorical(df, parent_var, df_name):
    """
    Aggregates the categorical features in a child dataframe
    for each observation of the parent variable.
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    parent_var : string
        The variable by which to group and aggregate the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with aggregated statistics for each observation of the parent_var
        The columns are also renamed and columns with duplicate values are removed.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[parent_var] = df[parent_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(parent_var).agg(['sum', 'count', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['sum', 'count', 'mean']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    # Remove duplicate columns by values
    _, idx = np.unique(categorical, axis = 1, return_index = True)
    categorical = categorical.iloc[:, idx]
    
    return categorical

In [15]:
bureau_categorical = agg_categorical(bureau, 'SK_ID_CURR', 'BUREAU')
bureau_categorical.head()

,BUREAU_CREDIT_TYPE_Mobile operator loan_mean,BUREAU_CREDIT_TYPE_Mobile operator loan_sum,BUREAU_CREDIT_TYPE_Loan for purchase of shares (margin lending)_mean,BUREAU_CREDIT_TYPE_Loan for purchase of shares (margin lending)_sum,BUREAU_CREDIT_ACTIVE_Bad debt_mean,BUREAU_CREDIT_ACTIVE_Bad debt_sum,BUREAU_CREDIT_TYPE_Interbank credit_mean,BUREAU_CREDIT_TYPE_Interbank credit_sum,BUREAU_CREDIT_TYPE_Real estate loan_mean,BUREAU_CREDIT_TYPE_Real estate loan_sum,...,BUREAU_CREDIT_TYPE_Credit card_sum,BUREAU_CREDIT_ACTIVE_Active_mean,BUREAU_CREDIT_ACTIVE_Closed_mean,BUREAU_CREDIT_TYPE_Consumer credit_mean,BUREAU_CREDIT_CURRENCY_currency 1_mean,BUREAU_CREDIT_ACTIVE_Active_sum,BUREAU_CREDIT_ACTIVE_Closed_sum,BUREAU_CREDIT_TYPE_Consumer credit_sum,BUREAU_CREDIT_CURRENCY_currency 1_sum,BUREAU_CREDIT_ACTIVE_Active_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.428571,0.571429,1.000000,1.0,3,4,7,7,7
100002.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,4,0.250000,0.750000,0.500000,1.0,2,6,4,8,8
100003.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,2,0.250000,0.750000,0.500000,1.0,1,3,2,4,4
100004.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.000000,1.000000,1.000000,1.0,0,2,2,2,2
100005.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1,0.666667,0.333333,0.666667,1.0,2,1,2,3,3


In [16]:
previous_categorical = agg_categorical(previous, 'SK_ID_CURR', 'PREVIOUS')
previous_categorical.head()

,PREVIOUS_NAME_GOODS_CATEGORY_Animals_mean,PREVIOUS_NAME_GOODS_CATEGORY_Animals_sum,PREVIOUS_NAME_GOODS_CATEGORY_House Construction_mean,PREVIOUS_NAME_GOODS_CATEGORY_House Construction_sum,PREVIOUS_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_mean,PREVIOUS_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_sum,PREVIOUS_NAME_CASH_LOAN_PURPOSE_Money for a third person_mean,PREVIOUS_NAME_CASH_LOAN_PURPOSE_Money for a third person_sum,PREVIOUS_NAME_CASH_LOAN_PURPOSE_Hobby_mean,PREVIOUS_NAME_CASH_LOAN_PURPOSE_Hobby_sum,...,PREVIOUS_CODE_REJECT_REASON_XAP_mean,PREVIOUS_FLAG_LAST_APPL_PER_CONTRACT_Y_mean,PREVIOUS_NAME_PORTFOLIO_POS_sum,PREVIOUS_NAME_CONTRACT_TYPE_Consumer loans_sum,PREVIOUS_NAME_CASH_LOAN_PURPOSE_XAP_sum,PREVIOUS_NAME_PRODUCT_TYPE_XNA_sum,PREVIOUS_NAME_CONTRACT_STATUS_Approved_sum,PREVIOUS_CODE_REJECT_REASON_XAP_sum,PREVIOUS_FLAG_LAST_APPL_PER_CONTRACT_Y_sum,PREVIOUS_NAME_CONTRACT_TYPE_Cash loans_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,1,1,1,1,1
100002.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,1,1,1,1,1
100003.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,2,2,2,2,3,3,3,3
100004.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,1,1,1,1,1
100005.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,2,1,2,2,2


The results from the two `agg_` functions can be merged with each other on the parent id (`SK_ID_CURR`). This dataframe can then be merged with the parent dataframe. It will have one row for every unique observation of the parent with the columns representing statistics aggregated across the parent's children. 


## Combined Aggregation Function

We can put these steps together into a function that will handle a child dataframe. The function will take care of both the numeric and categorical variables and will return the result of merging the two dataframes. 

In [17]:
import gc

def agg_child(df, parent_var, df_name):
    """Aggregate a child dataframe for each observation of the parent."""
    
    # Numeric and then categorical
    df_agg = agg_numeric(df, parent_var, df_name)
    df_agg_cat = agg_categorical(df, parent_var, df_name)
    
    # Merge on the parent variable
    df_info = df_agg.merge(df_agg_cat, on = parent_var, how = 'outer')
    
    # Remove any columns with duplicate values
    _, idx = np.unique(df_info, axis = 1, return_index = True)
    df_info = df_info.iloc[:, idx]
    
    # memory management
    gc.enable()
    del df_agg, df_agg_cat
    gc.collect()
    
    return df_info

In [18]:
bureau_results = agg_child(bureau, 'SK_ID_CURR', 'BUREAU')
bureau_results.head()

,BUREAU_DAYS_CREDIT_sum,BUREAU_DAYS_ENDDATE_FACT_sum,BUREAU_DAYS_CREDIT_min,BUREAU_DAYS_CREDIT_ENDDATE_min,BUREAU_DAYS_ENDDATE_FACT_min,BUREAU_DAYS_ENDDATE_FACT_mean,BUREAU_DAYS_CREDIT_mean,BUREAU_DAYS_CREDIT_UPDATE_sum,BUREAU_DAYS_ENDDATE_FACT_max,BUREAU_DAYS_CREDIT_UPDATE_min,...,BUREAU_AMT_CREDIT_SUM_DEBT_mean,BUREAU_AMT_CREDIT_SUM_min,BUREAU_AMT_CREDIT_SUM_mean,BUREAU_AMT_CREDIT_SUM_DEBT_max,BUREAU_AMT_CREDIT_SUM_max,BUREAU_AMT_CREDIT_SUM_DEBT_sum,BUREAU_AMT_CREDIT_SUM_sum,BUREAU_AMT_CREDIT_MAX_OVERDUE_min,BUREAU_AMT_CREDIT_MAX_OVERDUE_mean,BUREAU_AMT_CREDIT_MAX_OVERDUE_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,-5145.0,-3302.0,-1572.0,-1329.0,-1328.0,-825.500000,-735.000000,-652.0,-544.0,-155.0,...,85240.928571,85500.0,207623.571429,373239.0,378000.0,596686.5,1453365.000,NaN,NaN,NaN
100002.0,-6992.0,-4185.0,-1437.0,-1072.0,-1185.0,-697.500000,-874.000000,-3999.0,-36.0,-1185.0,...,49156.200000,0.0,108131.945625,245781.0,450000.0,245781.0,865055.565,0.0,1681.029,5043.645
100003.0,-5603.0,-3292.0,-2586.0,-2434.0,-2131.0,-1097.333333,-1400.750000,-3264.0,-540.0,-2131.0,...,0.000000,22248.0,254350.125000,0.0,810000.0,0.0,1017400.500,0.0,0.000,0.000
100004.0,-1734.0,-1065.0,-1326.0,-595.0,-683.0,-532.500000,-867.000000,-1064.0,-382.0,-682.0,...,0.000000,94500.0,94518.900000,0.0,94537.8,0.0,189037.800,0.0,0.000,0.000
100005.0,-572.0,-123.0,-373.0,-128.0,-123.0,-123.000000,-190.666667,-163.0,-123.0,-121.0,...,189469.500000,29826.0,219042.000000,543087.0,568800.0,568408.5,657126.000,0.0,0.000,0.000


Now instead of only making a handful of features from the `bureau` table one at a time, we have made 107 in a single function call! The __semi-automated approach does not take into account domain knowledge__ which has both its features and drawbacks: we make many more features in a reduced amount of time, but these might be completely irrelevant to the problem. 

This function can be applied to both `bureau` and `previous` because these are direct children of `app`. For the children of the children, we will need to take an additional aggregation step. 

# Aggregate Grandchild Data Tables

Several of the tables (`bureau_balance, cash, credit_card`, and `installments`) are children of the child dataframes. In other words, these are grandchildren of the main `app` data table. To aggregate these tables, they must first be aggregated at the parent level (which is on a per loan basis) and then at the grandparent level (which is on the client basis). For example, in the `bureau_balance` dataframe, there is monthly information on the loans in `bureau`. To get this data into the `app` dataframe will first require grouping the monthly information for each loan and then grouping the loans for each client. 

Hopefully, the nomenclature does not get too confusing, but here's a rounddown:

* __grandchild__: the child of a child data table, for instance, `bureau_balance`. For every row in the child table, there can be multiple rows in the grandchild. 
* __parent__: the parent table of the grandchild that links the grandchild to the grandparent. For example, the `bureau` dataframe is the parent of the `bureau_balance` dataframe in this situation. `bureau` is in turn the child of the `app` dataframe. `bureau_balance` can be connected to `app` through `bureau`.
* __grandparent__: the parent of the parent of the grandchild, in this problem the `app` dataframe. The end goal is to aggregate the information in the grandchild into the grandparent. This will be done in two stages: first at the parent (loan) level and then at the grandparent (client) level
* __parent variable__: the variable linking the grandchild to the parent. For the `bureau` and `bureau_balance` data this is `SK_ID_BUREAU` which uniquely identifies each previous loan
* __grandparent variable__: the variable linking the parent to the grandparent. This is `SK_ID_CURR` which uniquely identifies each client in `app`.

For an example, we will work through the `bureau_balance` dataframe. First we group the data by the loan id, `SK_ID_BUREAU` and aggregate information at the loan level. We already did this in traditional feature engineering, but this time we will not limit ourselves in the aggregations and the columns to which we apply the operations.

In [19]:
# Aggregate data at the loan level
bureau_balance_agg_loan = agg_numeric(bureau_balance, 'SK_ID_BUREAU', 'BB_LOAN')
bureau_balance_agg_loan.head()

,BB_LOAN_MONTHS_BALANCE_sum,BB_LOAN_MONTHS_BALANCE_min,BB_LOAN_MONTHS_BALANCE_mean,BB_LOAN_MONTHS_BALANCE_max,BB_LOAN_MONTHS_BALANCE_count
SK_ID_BUREAU,,,,,
5001709,-4656,-96,-48.0,0,97
5001710,-3403,-82,-41.0,0,83
5001711,-6,-3,-1.5,0,4
5001712,-171,-18,-9.0,0,19
5001713,-231,-21,-10.5,0,22


The resulting dataframe is aggregated at the `loan` level for every numeric variable. Now we need to aggregate this at the client level by repeating the `agg_numeric` calculation but this time with the `SK_ID_CURR` as the grouping variable. To get the grandparent variable back in the dataframe, we first have to merge with the `bureau` dataframe.

In [20]:
# Need to get the grandparent variable back in the data
bureau_balance_agg_loan = bureau_balance_agg_loan.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], 
                                                        on = 'SK_ID_BUREAU', how = 'left')

Now we can aggregate the loan level information for each client by grouping on `SK_ID_CURR` and calculating statistics.

In [21]:
# Aggregate the previous loans for each client
bureau_balance_agg_client = agg_numeric(bureau_balance_agg_loan, 'SK_ID_CURR', 'BB_CLIENT')
bureau_balance_agg_client.head()

,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_sum,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_min,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_min_sum,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_mean_sum,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_min_min,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_mean_min,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_min_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_mean_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_max,...,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_mean_max,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_max_sum,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_max_min,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_max_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_max_max,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_count_min,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_count,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_count_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_count_max,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_count_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,-2800,-1326,-400.000000,-165,-82.5,-51,-25.5,-23.571429,-11.785714,-1,...,-0.5,0,0,0.0,0,2,7,24.571429,52,172
100002.0,-2701,-632,-337.625000,-226,-175.0,-47,-39.5,-28.250000,-21.875000,-6,...,-1.5,-124,-32,-15.5,0,4,8,13.750000,22,110
100005.0,-91,-78,-30.333333,-18,-9.0,-12,-6.0,-6.000000,-3.000000,-3,...,-1.0,0,0,0.0,0,3,3,7.000000,13,21
100010.0,-3312,-2610,-1656.000000,-127,-92.0,-90,-72.5,-63.500000,-46.000000,-702,...,-19.5,-57,-55,-28.5,-2,36,2,36.000000,36,72
100013.0,-6756,-2346,-1689.000000,-226,-113.0,-68,-34.0,-56.500000,-28.250000,-780,...,-19.5,0,0,0.0,0,40,4,57.500000,69,230


For each observation of the grandparent variable, `SK_ID_CURR`, there is one row. This dataframe is now ready to be merged with the `app` data to be used for training and testing a model. 

Some of the resulting columns can be a little difficult to interpret because they are not neccessarily based on any meaningful physical quantity. As one example, `BB_CLIENT_BB_LOAN_MONTHS_BALANCE_count_mean` is the client's average number of records per previous loan. This is formed by `count`ing the number of `MONTHS_BALANCE` for each previous loan and then taking the `mean` of this across a clients loans. Another column is `BB_CLIENT_BB_LOAN_MONTHS_BALANCE_min_mean` which represents a client's average minimum `MONTHS_BALANCE` for each loan. That is, find the minimum `MONTHS_BALANCE` for each loan, and then average these across a client's previous loans. It is difficult to say ahead of time if any of these features will be useful but they can potentially capture information that is related to the problem.

### Categorical Variables for Grandchild Tables

We now repeat the same process of aggregating the grandchild at the grandparent level for the categorical variables. 
We already saw how for the `bureau` data, we could one-hot encode the categories and then calculate the `count`, `sum`, and `mean` for each each unique value in a category. The process for a grandchild table will proceed in the same way, but again, we will need to add an extra aggregation to encode the information on a client level. 

First we will find the categorical aggregations for `bureau_balance` at the loan level (grouping by the parent variable `SK_ID_BUREAU`).

In [22]:
bureau_balance_agg_cat_loan = agg_categorical(bureau_balance, 'SK_ID_BUREAU', 'BB_LOAN')
bureau_balance_agg_cat_loan.head()

,BB_LOAN_STATUS_4_mean,BB_LOAN_STATUS_3_mean,BB_LOAN_STATUS_2_mean,BB_LOAN_STATUS_5_mean,BB_LOAN_STATUS_4_sum,BB_LOAN_STATUS_3_sum,BB_LOAN_STATUS_2_sum,BB_LOAN_STATUS_5_sum,BB_LOAN_STATUS_1_mean,BB_LOAN_STATUS_1_sum,BB_LOAN_STATUS_0_mean,BB_LOAN_STATUS_0_sum,BB_LOAN_STATUS_X_mean,BB_LOAN_STATUS_C_mean,BB_LOAN_STATUS_X_sum,BB_LOAN_STATUS_C_sum,BB_LOAN_STATUS_0_count
SK_ID_BUREAU,,,,,,,,,,,,,,,,,
5001709,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.000000,0,0.113402,0.886598,11,86,97
5001710,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.060241,5,0.361446,0.578313,30,48,83
5001711,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.750000,3,0.250000,0.000000,1,0,4
5001712,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.526316,10,0.000000,0.473684,0,9,19
5001713,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.000000,0,1.000000,0.000000,22,0,22


Next we can use the `agg_numeric` function to group the information on a client level using the `SK_ID_CURR`. Again this first requires merging with the `bureau` data to get the `SK_ID_CURR`. 

In [23]:
# Merge to get the client id
bureau_balance_agg_cat_loan = bureau_balance_agg_cat_loan.merge(bureau[['SK_ID_CURR', 'SK_ID_BUREAU']], 
                                                                on = 'SK_ID_BUREAU', how = 'left')

# Aggregate the categorical information (which is now numeric) at the client level
bureau_balance_agg_cat_client = agg_numeric(bureau_balance_agg_cat_loan, 'SK_ID_CURR', 'BB_CLIENT')
bureau_balance_agg_cat_client.head()

,BB_CLIENT_BB_LOAN_STATUS_5_mean_min,BB_CLIENT_BB_LOAN_STATUS_5_sum_min,BB_CLIENT_BB_LOAN_STATUS_4_mean_min,BB_CLIENT_BB_LOAN_STATUS_4_sum_min,BB_CLIENT_BB_LOAN_STATUS_3_mean_min,BB_CLIENT_BB_LOAN_STATUS_3_sum_min,BB_CLIENT_BB_LOAN_STATUS_4_mean_mean,BB_CLIENT_BB_LOAN_STATUS_5_mean_mean,BB_CLIENT_BB_LOAN_STATUS_4_mean_max,BB_CLIENT_BB_LOAN_STATUS_4_mean_sum,...,BB_CLIENT_BB_LOAN_STATUS_X_sum_max,BB_CLIENT_BB_LOAN_STATUS_0_sum_max,BB_CLIENT_BB_LOAN_STATUS_C_sum_mean,BB_CLIENT_BB_LOAN_STATUS_0_count_mean,BB_CLIENT_BB_LOAN_STATUS_X_sum_sum,BB_CLIENT_BB_LOAN_STATUS_0_sum_sum,BB_CLIENT_BB_LOAN_STATUS_C_sum_max,BB_CLIENT_BB_LOAN_STATUS_0_count_max,BB_CLIENT_BB_LOAN_STATUS_C_sum_sum,BB_CLIENT_BB_LOAN_STATUS_0_count_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,...,9,12,15.714286,24.571429,30.0,31.0,44,52,110.0,172
100002.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,...,3,18,2.875000,13.750000,15.0,45.0,13,22,23.0,110
100005.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,...,1,7,1.666667,7.000000,2.0,14.0,5,13,5.0,21
100010.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,...,0,10,26.000000,36.000000,0.0,20.0,26,36,52.0,72
100013.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,...,40,34,25.750000,57.500000,41.0,79.0,44,69,103.0,230


Let's try to interpret one of these columns: `BB_CLIENT_BB_LOAN_STATUS_0_sum_mean`. This represents a client's average sum of `STATUS=0` across previous loans. That is, for each previous loan, sum up the total number of `STATUS` that are equal to 0, and then average this value across the loans of a client. In traditional feature engineering, we limited ourselves to specific values of the categorical features because we thought these were more important. The approach above makes no such presumptions.

To get all the information from the numeric and categorical variables from the monthly loan information at the client level, we need to join the resulting dataframes on the grandparent variable.

In [24]:
# Join numeric and categorical aggregations
bureau_balance_info = bureau_balance_agg_client.merge(bureau_balance_agg_cat_client, 
                                                      on = 'SK_ID_CURR', how = 'outer')

bureau_balance_info.head()

,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_sum,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_min,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_min_sum,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_mean_sum,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_min_min,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_mean_min,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_min_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_mean_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_max,...,BB_CLIENT_BB_LOAN_STATUS_X_sum_max,BB_CLIENT_BB_LOAN_STATUS_0_sum_max,BB_CLIENT_BB_LOAN_STATUS_C_sum_mean,BB_CLIENT_BB_LOAN_STATUS_0_count_mean,BB_CLIENT_BB_LOAN_STATUS_X_sum_sum,BB_CLIENT_BB_LOAN_STATUS_0_sum_sum,BB_CLIENT_BB_LOAN_STATUS_C_sum_max,BB_CLIENT_BB_LOAN_STATUS_0_count_max,BB_CLIENT_BB_LOAN_STATUS_C_sum_sum,BB_CLIENT_BB_LOAN_STATUS_0_count_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,-2800,-1326,-400.000000,-165,-82.5,-51,-25.5,-23.571429,-11.785714,-1,...,9,12,15.714286,24.571429,30.0,31.0,44,52,110.0,172
100002.0,-2701,-632,-337.625000,-226,-175.0,-47,-39.5,-28.250000,-21.875000,-6,...,3,18,2.875000,13.750000,15.0,45.0,13,22,23.0,110
100005.0,-91,-78,-30.333333,-18,-9.0,-12,-6.0,-6.000000,-3.000000,-3,...,1,7,1.666667,7.000000,2.0,14.0,5,13,5.0,21
100010.0,-3312,-2610,-1656.000000,-127,-92.0,-90,-72.5,-63.500000,-46.000000,-702,...,0,10,26.000000,36.000000,0.0,20.0,26,36,52.0,72
100013.0,-6756,-2346,-1689.000000,-226,-113.0,-68,-34.0,-56.500000,-28.250000,-780,...,40,34,25.750000,57.500000,41.0,79.0,44,69,103.0,230


For each client, we now have 90 features that were created from the `bureau_balance` data alone. To get this in the final dataframe, we would merge with app on the `SK_ID_CURR`. 

## Function for Aggregating Grandchildren

We can take the individual steps required for aggregating a grandchild dataframe at the grandparent level in a function. These are:

1. Aggregate the numeric variables at the parent (the loan, `SK_ID_BUREAU` or `SK_ID_PREV`) level.
2. Merge with the parent of the grandchild to get the grandparent variable in the data (for example `SK_ID_CURR`)
3. Aggregate the numeric variables at the grandparent (the client, `SK_ID_CURR`) level. 
4. Aggregate the categorical variables at the parent level.
5. Merge the aggregated data with the parent to get the grandparent variable
6. Aggregate the categorical variables at the grandparent level
7. Merge the numeric and categorical dataframes on the grandparent varible
8. Remove the columns with all duplicated values.
9. The resulting dataframe should now have one row for every grandparent (client) observation
10. Merge with the main dataframe (`app`) on the grandparent variable (`SK_ID_CURR`). 

This function can be applied to __all 4 grandchildren__ without the need for hard-coding in specific variables. The advantages of functions are re-usability, accuracy, and efficiency.

In [25]:
# memory management
import gc

def agg_grandchild(df, parent_df, parent_var, grandparent_var, df_name):
    """
    Aggregate a grandchild dataframe at the grandparent level.
    
    Parameters
    --------
        df : dataframe
            Data with each row representing one observation
            
        parent_df : dataframe
            Parent table of df that must have the parent_var and 
            the grandparent_var. Used only to get the grandparent_var into
            the dataframe after aggregations
            
        parent_var : string
            Variable representing each unique observation in the parent.
            For example, `SK_ID_BUREAU` or `SK_ID_PREV`
            
        grandparent_var : string
            Variable representing each unique observation in the grandparent.
            For example, `SK_ID_CURR`. 
            
        df_name : string
            String for renaming the resulting columns.
            The columns are name with the `df_name` and with the 
            statistic calculated in the column
    
    Return
    --------
        df_info : dataframe
            A dataframe with one row for each observation of the grandparent variable.
            The grandparent variable forms the index, and the resulting dataframe
            can be merged with the grandparent to be used for training/testing. 
            Columns with all duplicate values are removed from the dataframe before returning.
    
    """
    
    # set the parent_var as the index of the parent_df for faster merges
    parent_df = parent_df[[parent_var, grandparent_var]].copy().set_index(parent_var)
    
    # Aggregate the numeric variables at the parent level
    df_agg = agg_numeric(df, parent_var, '%s_LOAN' % df_name)
    
    # Merge to get the grandparent variable in the data
    df_agg = df_agg.merge(parent_df, 
                          on = parent_var, how = 'left')
    
    # Aggregate the numeric variables at the grandparent level
    df_agg_client = agg_numeric(df_agg, grandparent_var, '%s_CLIENT' % df_name)
    
    # Can only apply one-hot encoding to categorical variables
    if any(df.dtypes == 'object'):
    
        # Aggregate the categorical variables at the parent level
        df_agg_cat = agg_categorical(df, parent_var, '%s_LOAN' % df_name)
        df_agg_cat = df_agg_cat.merge(parent_df,
                                      on = parent_var, how = 'left')

        # Aggregate the categorical variables at the grandparent level
        df_agg_cat_client = agg_numeric(df_agg_cat, grandparent_var, '%s_CLIENT' % df_name)
        df_info = df_agg_client.merge(df_agg_cat_client, on = grandparent_var, how = 'outer')
        
        gc.enable()
        del df_agg, df_agg_client, df_agg_cat, df_agg_cat_client
        gc.collect()
    
    # If there are no categorical variables, then we only need the numeric aggregations
    else:
        df_info = df_agg_client.copy()
    
        gc.enable()
        del df_agg, df_agg_client
        gc.collect()
    
    # Drop the columns with all duplicated values
    _, idx = np.unique(df_info, axis = 1, return_index=True)
    df_info = df_info.iloc[:, idx]
    
    return df_info

In [27]:
bureau_balance_results = agg_grandchild(bureau_balance, bureau, 'SK_ID_BUREAU', 'SK_ID_CURR', 'BB')
bureau_balance_results.head()

,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_sum,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_min,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_min_sum,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_mean_sum,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_min_min,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_mean_min,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_min_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_mean_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_sum_max,...,BB_CLIENT_BB_LOAN_STATUS_X_sum_max,BB_CLIENT_BB_LOAN_STATUS_0_sum_max,BB_CLIENT_BB_LOAN_STATUS_C_sum_mean,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_count_mean,BB_CLIENT_BB_LOAN_STATUS_X_sum_sum,BB_CLIENT_BB_LOAN_STATUS_0_sum_sum,BB_CLIENT_BB_LOAN_STATUS_C_sum_max,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_count_max,BB_CLIENT_BB_LOAN_STATUS_C_sum_sum,BB_CLIENT_BB_LOAN_MONTHS_BALANCE_count_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,-2800,-1326,-400.000000,-165,-82.5,-51,-25.5,-23.571429,-11.785714,-1,...,9,12,15.714286,24.571429,30.0,31.0,44,52,110.0,172
100002.0,-2701,-632,-337.625000,-226,-175.0,-47,-39.5,-28.250000,-21.875000,-6,...,3,18,2.875000,13.750000,15.0,45.0,13,22,23.0,110
100005.0,-91,-78,-30.333333,-18,-9.0,-12,-6.0,-6.000000,-3.000000,-3,...,1,7,1.666667,7.000000,2.0,14.0,5,13,5.0,21
100010.0,-3312,-2610,-1656.000000,-127,-92.0,-90,-72.5,-63.500000,-46.000000,-702,...,0,10,26.000000,36.000000,0.0,20.0,26,36,52.0,72
100013.0,-6756,-2346,-1689.000000,-226,-113.0,-68,-34.0,-56.500000,-28.250000,-780,...,40,34,25.750000,57.500000,41.0,79.0,44,69,103.0,230


# Putting it Together

Now that we have the individual pieces of semi-automated feature engineering, we need to put them together. There are two functions that can handle the children and the grandchildren data tables:

1. `agg_child(df, parent_var, df_name)`: aggregate the numeric and categorical variables of a child dataframe at the parent level. For example, the `previous` dataframe is a child of the `app` dataframe that must be aggregated for each client. 
2. `agg_grandchild(df, parent_df, parent_var, grandparent_var, df_name)`: aggregate the numeric and categorical variables of a grandchild dataframe at the grandparent level. For example, the `bureau_balance` dataframe is the grandchild of the `app` dataframe with `bureau` as the parent. 

For each of the children dataframes of `app`, (`previous` and `bureau`), we will use the first function and merge the result into the `app` on the parent variable, `SK_ID_CURR`. For the four grandchild dataframes, we will use the second function, which returns a single dataframe that can then be merged into app on `SK_ID_CURR`. 

#### Bureau

In [ ]:
bureau = pd.read_csv('../input/bureau.csv')
bureau_info = agg_child(bureau, 'SK_ID_CURR', 'BUREAU')
bureau_info.shape

#### Previous

In [ ]:
previous = pd.read_csv('../input/previous_application.csv')
previous_info = agg_child(previous, 'SK_ID_CURR', 'PREVIOUS')
previous_info.shape

#### Bureau Balance

In [ ]:
bureau_balance = pd.read_csv('../input/bureau_balance.csv')
bureau_balance_info = agg_grandchild(bureau_balance, bureau, 'SK_ID_BUREAU', 'SK_ID_CURR', 'BB')
del bureau_balance
bureau_balance_info.shape

#### Credit Card Monthly Data

In [ ]:
credit_card = pd.read_csv('../input/credit_card_balance.csv')
credit_card_info = agg_grandchild(credit_card, previous, 'SK_ID_PREV', 'SK_ID_CURR', 'CC')
del credit_card
credit_card_info.shape

#### POS Monthly Data

In [ ]:
cash = pd.read_csv('../input/POS_CASH_balance.csv')
cash_info = agg_grandchild(cash, previous, 'SK_ID_PREV', 'SK_ID_CURR', 'CASH')
del cash
cash_info.shape

#### Previous Installments

In [ ]:
installments = pd.read_csv('../input/installments_payments.csv')
installments_info = agg_grandchild(installments, previous, 'SK_ID_PREV', 'SK_ID_CURR', 'IN')
del installments
installments_info.shape

In [ ]:
gc.enable()
del bureau, previous
gc.collect()

### Merge Results into Main Dataframe

The individual dataframes can all be merged into the main `app` dataframe. Merging is much quicker if done on any index, so it's good practice to first set the index to the variable on which we will merge. In each case, we use a `left` join so that all the observations in `app` are kept even if they are not present in the other dataframes (which occurs because not every client has previous records at Home Bureau or other credit institutions). 

The final result is one dataframe with a single row for each client that can be used for training a machine learning model. 

In [ ]:
app = app.set_index('SK_ID_CURR')
app = app.merge(bureau_info, on = 'SK_ID_CURR', how = 'left')
del bureau_info
app.shape

In [ ]:
app = app.merge(previous_info, on = 'SK_ID_CURR', how = 'left')
del previous_info
app.shape

In [ ]:
app = app.merge(bureau_balance_info, on = 'SK_ID_CURR', how = 'left')
del bureau_balance_info
app.shape

In [ ]:
app = app.merge(credit_card_info, on = 'SK_ID_CURR', how = 'left')
del credit_card_info
app.shape

In [ ]:
app = app.merge(cash_info, on = 'SK_ID_CURR', how = 'left')
del cash_info
app.shape

In [ ]:
app = app.merge(installments_info, on = 'SK_ID_CURR', how = 'left')
del installments_info
app.shape

In [ ]:
print('After semi-automated feature engineering, there are {} features.'.format(app.shape[1] - 2))

In [ ]:
# Check for columns with duplicated values
_, idx = np.unique(app, axis = 1, return_index = True)
print('There are {} columns with all duplicated values.'.format(app.shape[1] - len(idx)))

Many of the new features will likely not be useful, but it's nearly impossible to say before modeling which these are. We can apply feature selection procedures such as removing highly correlated (collinear) features or removing features with a missing percentage greater than a certain threshold. We can also use modeling results in the form of features importances to select only the n-most important features or the features comprising a certain percentage of the total feature importance.

In [ ]:
app = app.reset_index()
# app.to_csv('../input/final_manual_features.csv', index = False)

### Cross-Validation Score

To get a sense of how much the semi-automated features help, we can train the model with default hyperparameters (using early stopping with cross validation to select the number of estimators). This is not the best indicator of the usefulness of the features because model hyperparameter tuning can make a significant difference (especially with a large number of features where regularization becomes critical) but it will allow for a comparison with the baseline and with the model trained only on the traditional manual feature engineering data.

In [ ]:
# Extract the training and testing data
train, test = app[app['TARGET'].notnull()], app[app['TARGET'].isnull()]

cv_results = cross_validate(train)

## Cross Validation Results

The semi-automated feature modeling did improve the cross validation score. The cross-validation scores are as follows:

* Baseline: 0.75565
* Traditional Manual Feature Engineering: 0.77227
* Semi-Automated + Traditional Manual Feature Engineering: 0.77987

This is not a completely fair comparison because we have not tuned the hyperparameters of the gradient boosting machine. However, it does give us confidence that the feature engineering steps were able to improve the model performance. Further work will require model tuning using random search or automated tuning methods such as Bayesian optimization. 

To find the feature importances, we have to train another model (`cv` does not return the feature importances). 

In [ ]:
def plot_feature_importances(df):
    """
    Plots 15 most important features and returns a sorted feature importance dataframe.
    
    Parameters
    --------
    df : dataframe
        Dataframe of feature importances. Columns must be feature and importance

        
    Return
    --------
    df : dataframe
        Dataframe ordered by feature importances with a normalized column (sums to 1)
        and a cumulative importance column
    
    """
    
    plt.rcParams['font.size'] = 18
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()
    df['cumulative_importance'] = np.cumsum(df['importance_normalized'])

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()

    
    return df

In [ ]:
train = train.drop(columns = ['SK_ID_CURR', 'TARGET'])
feature_names = train.columns

test = test.drop(columns = ['SK_ID_CURR', 'TARGET'])

# Fit using the best number of estimators from cross validation
model = lgb.LGBMClassifier(n_estimators = len(cv_results['auc-mean']), random_state=RSEED)
model.fit(train, train_labels)

# Extract the feature importances
fi = pd.DataFrame({'feature': feature_names, 
                   'importance': model.feature_importances_})

In [ ]:
submission_features = make_submission(cv_results, train, test)
submission_features.to_csv('../input/submission_features.csv', index = False)

In [ ]:
norm_fi = plot_feature_importances(fi)
norm_fi.head(10)

These results should give us confidence that our feature engineering - both traditional and semi-automated - was worthwhile. The cross validation results increased over the baseline model and a number of the top ten most important features were derived from manual feature engineering.

# Conclusions 

Semi-automated feature engineering is a more efficient approach than traditional manual feature engineering. By writing functions rather than hand-coding each feature, we can add hundreds of features at once and re-use the functions for multiple tables. In comparison to manual feature engineering, the semi-automated approach is quicker (in terms of time per feature), less error-prone, and more repeatable across datasets. Overall, semi-automated feature engineering is a much more efficient process although we may lose a little in terms of domain knowledge in the resulting features. Moreover, if we are going to the trouble of semi-automated feature engineering, why not take the step to full automation?
 
The final metrics of manual, semi-automated, and fully automated (implemented in the Automated notebook) features are as follows:

| Dataset                     	| Total Features 	| Time Spent (conservative estimate) 	| CV ROC AUC 	| Public Leaderboard ROC AUC 	|
|-----------------------------	|----------------	|------------------------------------	|------------	|----------------------------	|
| Main after one-hot encoding 	| 241            	| 15 minutes                         	| 0.75565    	| 0.741                      	|
| Manual Feature Engineering  	| 271            	| 10 hours 	| 0.77227    	| 0.786                      	|
| Automated Feature Engineering  	| 2109 | 1 hour 	| 0.77475     	| 0.787                      	|
| Manual + Semi-Automated     	| 1444           	| 20 hours 	| 0.77987    	| 0.791                      	|

The takeaways from this notebook are:

* Manual feature engineering: building features one at a time using domain knowledge and work done by other data scientists on the problem
* Semi-automated feature engineering: write functions that build new features using aggregations without any consideration of the relevance to the problem

Both methods have their advantages and drawbacks. __The traditional method resulted in 30 new features, with an average time to make each feature of over 15 miuntes. These features turned out to be very important in the final model. The semi-automated approach created 1173 features with about 10 total hours of work on the functions. This represents a time per feature of 0.51 minutes. The much larger number of features must be weighed against the usefulness of these features as many of them have no importance to the model.__

The next notebook will go over an even more efficient approach: fully automated feature engineering using Featuretools. With this method, we can make thousands of features from a single function call, and only have to keep track of one data structure! 

For the implementation of automated feature engineering using Featuretools, take a look at the Automated Loan Repayment notebook. The Results notebook summarizes the final metrics from each method.